# Churn Prediction

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Churn Prediction').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
data = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [4]:
data.printSchema()
data.columns

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
assembler = VectorAssembler(
    inputCols=[ 'Age', 'Total_Purchase', 'Years', 'Num_Sites'],
    outputCol = 'features')

mldata = assembler.transform(data)

In [7]:
mldata.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- features: vector (nullable = true)



In [8]:
transdata = mldata.select('features','churn')

In [9]:
train, test = transdata.randomSplit([0.7,0.3])

In [10]:
lr = LogisticRegression(labelCol='churn', featuresCol='features')
lrModel = lr.fit(train)

In [11]:
lrModel.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                630|                630|
|   mean|0.16825396825396827|0.12857142857142856|
| stddev|0.37438891251493644| 0.3349909575357319|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [13]:
testmodel = lrModel.evaluate(test)

In [14]:
testmodel.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8787.39,5.4...|    1|[0.90985155695656...|[0.71296978575373...|       0.0|
|[28.0,11128.95,5....|    0|[4.51011250108475...|[0.98912240047990...|       0.0|
|[28.0,11204.23,3....|    0|[1.72489267596746...|[0.84875797104831...|       0.0|
|[28.0,11245.38,6....|    0|[3.48354878356733...|[0.97021604025388...|       0.0|
|[29.0,5900.78,5.5...|    0|[4.56407001177733...|[0.98968788299623...|       0.0|
|[29.0,10203.18,5....|    0|[4.07584620850243...|[0.98330559319520...|       0.0|
|[29.0,13240.01,4....|    0|[7.10478157125512...|[0.99917950472269...|       0.0|
|[30.0,7960.64,2.7...|    1|[3.68973795548619...|[0.97563017610771...|       0.0|
|[30.0,8874.83,5.5...|    0|[3.04849117772869...|[0.95471734163713...|       0.0|
|[30.0,10960.52,

In [15]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [16]:
areaUnderCurve = churn_eval.evaluate(testmodel.predictions)

In [17]:
areaUnderCurve

0.7389380530973452

In [18]:
LRModel = lr.fit(transdata)

In [19]:
#loading teh new data
customerdata = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [20]:
customermldata = assembler.transform(customerdata)

In [21]:
customermldata.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
customerChurnResult = LRModel.transform(customermldata)

In [23]:
customerChurnResult.select('company','prediction').show()

+----------------+----------+
|         company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

